# 1. Download SP500 Price Data

## 1.1 Import all necessary libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplt
import statsmodels.api as sm
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
from statsmodels.regression.rolling import RollingOLS

In [3]:
warnings.filterwarnings('ignore')

## 1.2 Download the SP500 Constituent Data

In [4]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
print(type(sp500[0]))


<class 'pandas.core.frame.DataFrame'>


## 1.3 Clean the Data

- Isolate the ticker symbols
- Store the isoalted ticker symbols into a list

In [5]:
tickers_list = sp500[0]["Symbol"].tolist()

## 1.4 Take the start and end date to determine a range

- For this example, I will use 8 years

In [6]:
end_date = dt.datetime.today()
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

## 1.5.1 Per ticker, download the necessary information from the start date to the end date

In [7]:
df = yf.download(tickers=tickers_list, 
                 start = start_date, 
                 end = end_date).stack()
df

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2016-05-11 19:30:47.770525 -> 2024-05-09 19:30:47.770525)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2016-05-11 A        39.843704   42.459999   43.200001   42.119999   42.189999   
           AAL      31.579227   32.860001   34.040001   32.849998   33.680000   
           AAPL     21.271112   23.127501   23.392500   23.115000   23.370001   
           ABBV     44.434517   62.700001   63.830002   62.639999   63.770000   
           ABT      32.806599   38.009998   38.290001   37.900002   38.200001   
...                       ...         ...         ...         ...         ...   
2024-05-08 XYL     141.199997  141.199997  141.300003  139.750000  139.919998   
           YUM     136.960007  136.960007  137.320007  135.479996  135.809998   
           ZBH     120.849998  120.849998  121.290001  120.000000  121.120003   
           ZBRA    319.950012  319.950012  320.059998  312.190002  314.940002   
           ZTS     164.919998  164.919998  167.500000  161.000000  167.080002   

Price                   Volume  
Date       Ticker               
2016-05-11 A         3329000.0  
           AAL       8148600.0  
           AAPL    114876400.0  
           ABBV      9375200.0  
           ABT      10598300.0  
...                        ...  
2024-05-08 XYL        894400.0  
           YUM       1891000.0  
           ZBH       1256700.0  
           ZBRA       329800.0  
           ZTS       4673100.0  

[988194 rows x 6 columns]


# 1.5.2 Make Data More Readable

In [8]:
df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-11 A        39.843704   42.459999   43.200001   42.119999   42.189999   
           AAL      31.579227   32.860001   34.040001   32.849998   33.680000   
           AAPL     21.271112   23.127501   23.392500   23.115000   23.370001   
           ABBV     44.434517   62.700001   63.830002   62.639999   63.770000   
           ABT      32.806599   38.009998   38.290001   37.900002   38.200001   
...                       ...         ...         ...         ...         ...   
2024-05-08 XYL     141.199997  141.199997  141.300003  139.750000  139.919998   
           YUM     136.960007  136.960007  137.320007  135.479996  135.809998   
           ZBH     120.849998  120.849998  121.290001  120.000000  121.120003   
           ZBRA    319.950012  319.950012  320.059998  312.190002  314.940002   
           ZTS     164.919998  164.919998  167.500000  161.000000  167.080002   

Price                   volume  
date       ticker               
2016-05-11 A         3329000.0  
           AAL       8148600.0  
           AAPL    114876400.0  
           ABBV      9375200.0  
           ABT      10598300.0  
...                        ...  
2024-05-08 XYL        894400.0  
           YUM       1891000.0  
           ZBH       1256700.0  
           ZBRA       329800.0  
           ZTS       4673100.0  

[988194 rows x 6 columns]

In [9]:
df.to_csv("x.csv")

# 2. Calculate Different Technical Indicators per Stock

- RSI
- Bollinger Bands
- MACD
- ATR
- Garman-Klass Volatility
- Dollar Volume

In [10]:
def compute_atr(tickers_list):
    atr = pandas_ta.atr(high = tickers_list['high'],
                        low = tickers_list['low'],
                        close = tickers_list['close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std())   

In [11]:


def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())


In [22]:
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length= 20))

df['bb_low'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,0])

df['bb_mid'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,1])

df['bb_upper'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,2])

df ['atr'] = df.groupby (level = 1, group_keys = False).apply(compute_atr)

df['ema12'] = df['close'].ewm(span = 12, adjust = False).mean()

df['ema26'] = df['close'].ewm(span = 26, adjust = False).mean()

df['macd'] = df['ema12'] - df['ema26']

df['dollar_volume'] = (df['adj close'] * df['volume']) / 1e6

df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

In [13]:
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-11 A        39.843704   42.459999   43.200001   42.119999   42.189999   
           AAL      31.579227   32.860001   34.040001   32.849998   33.680000   
           AAPL     21.271112   23.127501   23.392500   23.115000   23.370001   
           ABBV     44.434517   62.700001   63.830002   62.639999   63.770000   
           ABT      32.806599   38.009998   38.290001   37.900002   38.200001   
...                       ...         ...         ...         ...         ...   
2024-05-08 XYL     141.199997  141.199997  141.300003  139.750000  139.919998   
           YUM     136.960007  136.960007  137.320007  135.479996  135.809998   
           ZBH     120.849998  120.849998  121.290001  120.000000  121.120003   
           ZBRA    319.950012  319.950012  320.059998  312.190002  314.940002   
           ZTS     164.919998  164.919998  167.500000  161.000000  167.080002   

Price                   volume        rsi    bb_low    bb_mid  bb_upper  \
date       ticker                                                         
2016-05-11 A         3329000.0        NaN       NaN       NaN       NaN   
           AAL       8148600.0        NaN       NaN       NaN       NaN   
           AAPL    114876400.0        NaN       NaN       NaN       NaN   
           ABBV      9375200.0        NaN       NaN       NaN       NaN   
           ABT      10598300.0        NaN       NaN       NaN       NaN   
...                        ...        ...       ...       ...       ...   
2024-05-08 XYL        894400.0  75.279344  4.821461  4.887789  4.954116   
           YUM       1891000.0  48.681161  4.902432  4.937385  4.972337   
           ZBH       1256700.0  45.293740  4.772924  4.806016  4.839108   
           ZBRA       329800.0  64.326502  5.569790  5.685781  5.801772   
           ZTS       4673100.0  50.667098  4.967978  5.059545  5.151113   

Price                   atr       ema12       ema26       macd  dollar_volume  \
date       ticker                                                               
2016-05-11 A            NaN   42.459999   42.459999   0.000000     132.639691   
           AAL          NaN   40.983076   41.748888  -0.765812     257.326493   
           AAPL         NaN   38.236065   40.369526  -2.133461    2443.548821   
           ABBV         NaN   41.999747   42.023635  -0.023888     416.582483   
           ABT          NaN   41.385940   41.726329  -0.340389     347.694175   
...                     ...         ...         ...        ...            ...   
2024-05-08 XYL     0.796707  119.823961  130.289009 -10.465048     126.289277   
           YUM     0.539243  122.460276  130.783157  -8.322881     258.991373   
           ZBH    -0.280787  122.212541  130.047367  -7.834827     151.872193   
           ZBRA    0.405384  152.633690  144.114230   8.519460     105.519514   
           ZTS     1.522019  154.523891  145.655398   8.868493     770.687643   

Price              garman_klass_vol  
date       ticker                    
2016-05-11 A              -0.000944  
           AAL            -0.000969  
           AAPL           -0.003350  
           ABBV           -0.050239  
           ABT            -0.008897  
...                             ...  
2024-05-08 XYL             0.000029  
           YUM             0.000064  
           ZBH             0.000055  
           ZBRA            0.000214  
           ZTS             0.000718  

[988194 rows x 16 columns]

# 3. Aggregate on Monthly Level and Filter per Month the Most Liquid Stocks

In [15]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', ' volume', 'open',
                                                         'high', ' low', 'close']]


data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
        df.unstack()[last_cols].resample('M').last().stack('ticker')], 
        axis = 1)).dropna()

data

dollar_volume   adj close         low       volume  \
date       ticker                                                       
2016-06-30 A           99.655612   41.736317   43.590000    2181100.0   
           AAL        351.266370   27.206568   27.490000   11148300.0   
           AAPL      3133.495470   21.981602   23.575001  143345600.0   
           ABBV       418.335104   43.874649   61.139999    7920600.0   
           ABT        338.847601   33.928627   38.570000   10425000.0   
...                          ...         ...         ...          ...   
2024-05-31 GEHC       256.796843   81.110001   80.250000    2747700.0   
           KVUE       429.171165   20.450001   19.990000   24428500.0   
           VLTO        99.090988   96.730003   96.279999    1039000.0   
           GEV        573.178541  164.000000  163.225006    3679300.0   
           SOLV       113.424523   66.430000   65.150002    1910600.0   

                         rsi    bb_low    bb_mid  bb_upper       atr  \
date       ticker                                                      
2016-06-30 A       49.617733  3.721142  3.769610  3.818078 -1.173322   
           AAL     41.290580  3.253283  3.392594  3.531905  0.682303   
           AAPL    50.157332  3.102071  3.143204  3.184338 -1.206140   
           ABBV    52.419466  3.744297  3.788811  3.833325 -0.986066   
           ABT     56.558209  3.488460  3.527148  3.565836 -1.155338   
...                      ...       ...       ...       ...       ...   
2024-05-31 GEHC    43.719172  4.351377  4.437916  4.524454  0.955666   
           KVUE    60.684397  2.957388  3.001100  3.044812 -0.712313   
           VLTO    65.735541  4.473790  4.538882  4.603973 -0.147920   
           GEV     64.703608  4.831737  4.999259  5.166781 -0.147537   
           SOLV    52.682426  4.124473  4.170615  4.216757 -1.831488   

                        ema12       ema26       macd  garman_klass_vol  
date       ticker                                                       
2016-06-30 A        60.551534   63.273481  -2.721946         -0.000631  
           AAL      55.591298   60.683593  -5.092295          0.000214  
           AAPL     50.715714   57.958882  -7.243169         -0.001853  
           ABBV     52.437912   58.251558  -5.813646         -0.045187  
           ABT      50.418233   56.848480  -6.430246         -0.006377  
...                       ...         ...        ...               ...  
2024-05-31 GEHC    148.515833  160.603278 -12.087444          0.000133  
           KVUE    101.473245  123.933856 -22.460611          0.000447  
           VLTO    195.591796  204.292879  -8.701084          0.000021  
           GEV     134.170745  151.212687 -17.041942          0.000136  
           SOLV    236.534622  222.362511  14.172111          0.000249  

[47155 rows x 13 columns]

## 3.1 Compute the 5-year rolling average of the dollar volume per stock

In [44]:
data_clone = pd.DataFrame(df)

print(data_clone.groupby(level=1).head())

data_clone["ma_9"] = df['close'].rolling(9).mean()

data_clone["ma_9"] = df['close'].rolling(21).mean()

data_clone

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-11 A        39.843704   42.459999   43.200001   42.119999   42.189999   
           AAL      31.579227   32.860001   34.040001   32.849998   33.680000   
           AAPL     21.271112   23.127501   23.392500   23.115000   23.370001   
           ABBV     44.434517   62.700001   63.830002   62.639999   63.770000   
           ABT      32.806599   38.009998   38.290001   37.900002   38.200001   
...                       ...         ...         ...         ...         ...   
2024-04-03 SOLV     63.889999   63.889999   67.769997   63.860001   65.529999   
2024-04-04 GEV     126.059998  126.059998  139.350006  125.260002  136.000000   
           SOLV     67.000000   67.000000   68.165001   62.330002   63.049999   
2024-04-05 GEV     122.699997  122.699997  128.179993  119.000000  125.500000   
           SOLV     70.05999

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-11 A        39.843704   42.459999   43.200001   42.119999   42.189999   
           AAL      31.579227   32.860001   34.040001   32.849998   33.680000   
           AAPL     21.271112   23.127501   23.392500   23.115000   23.370001   
           ABBV     44.434517   62.700001   63.830002   62.639999   63.770000   
           ABT      32.806599   38.009998   38.290001   37.900002   38.200001   
...                       ...         ...         ...         ...         ...   
2024-05-08 XYL     141.199997  141.199997  141.300003  139.750000  139.919998   
           YUM     136.960007  136.960007  137.320007  135.479996  135.809998   
           ZBH     120.849998  120.849998  121.290001  120.000000  121.120003   
           ZBRA    319.950012  319.950012  320.059998  312.190002  314.940002   
           ZTS     164.919998  164.919998  167.500000  161.000000  167.080002   

Price                   volume        rsi    bb_low    bb_mid  bb_upper  \
date       ticker                                                         
2016-05-11 A         3329000.0        NaN       NaN       NaN       NaN   
           AAL       8148600.0        NaN       NaN       NaN       NaN   
           AAPL    114876400.0        NaN       NaN       NaN       NaN   
           ABBV      9375200.0        NaN       NaN       NaN       NaN   
           ABT      10598300.0        NaN       NaN       NaN       NaN   
...                        ...        ...       ...       ...       ...   
2024-05-08 XYL        894400.0  75.279344  4.821461  4.887789  4.954116   
           YUM       1891000.0  48.681161  4.902432  4.937385  4.972337   
           ZBH       1256700.0  45.293740  4.772924  4.806016  4.839108   
           ZBRA       329800.0  64.326502  5.569790  5.685781  5.801772   
           ZTS       4673100.0  50.667098  4.967978  5.059545  5.151113   

Price                   atr       ema12       ema26       macd  dollar_volume  \
date       ticker                                                               
2016-05-11 A            NaN   42.459999   42.459999   0.000000     132.639691   
           AAL          NaN   40.983076   41.748888  -0.765812     257.326493   
           AAPL         NaN   38.236065   40.369526  -2.133461    2443.548821   
           ABBV         NaN   41.999747   42.023635  -0.023888     416.582483   
           ABT          NaN   41.385940   41.726329  -0.340389     347.694175   
...                     ...         ...         ...        ...            ...   
2024-05-08 XYL     0.796707  119.823961  130.289009 -10.465048     126.289277   
           YUM     0.539243  122.460276  130.783157  -8.322881     258.991373   
           ZBH    -0.280787  122.212541  130.047367  -7.834827     151.872193   
           ZBRA    0.405384  152.633690  144.114230   8.519460     105.519514   
           ZTS     1.522019  154.523891  145.655398   8.868493     770.687643   

Price              garman_klass_vol        ma_9  
date       ticker                                
2016-05-11 A              -0.000944         NaN  
           AAL            -0.000969         NaN  
           AAPL           -0.003350         NaN  
           ABBV           -0.050239         NaN  
           ABT            -0.008897         NaN  
...                             ...         ...  
2024-05-08 XYL             0.000029  112.800952  
           YUM             0.000064  117.442857  
           ZBH             0.000055  115.313810  
           ZBRA            0.000214  115.130478  
           ZTS             0.000718  122.163335  

[988194 rows x 17 columns]

# 4. Calculate Monthly Returns for Different Time-Horizons

# 5. Download Fama-French Factors; Calculate Rolling Factor Betas Per Stock

# 6.  Per Month, Create a K-means Clustering Model to group similar assets based on their Features

# 7. Per Month, Select Assets Based on the Cluster and Build a Portfolio Based on That

# 8. Visualize the Portfolio Returns and Compare That with the SP500 